In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import HeteroData
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import os
from tqdm import tqdm
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class EnhancedDataProcessor:
    """데이터 전처리를 위한 클래스"""
    def __init__(self):
        self.visit_scaler = RobustScaler()
        self.travel_scaler = StandardScaler()
        # 제외할 키워드 목록
        self.exclude_keywords = {
            '역', '터미널', '공항', '휴게소', '정류장', '톨게이트', '교차로', '출구', '입구',
            'IC', 'JC', '나들목', '분기점', '요금소', '주차장', '주유소', '충전소',
            '아파트', '원룸', '오피스텔', '빌라', '주택', '빌딩', '상가', '모텔', '집', 
            '교직원', '하나로마트', '마트'
        }
        
    def should_exclude_location(self, name):
        """위치를 제외해야 하는지 확인"""
        if pd.isna(name):
            return False
        name_str = str(name).lower()
        
        for keyword in self.exclude_keywords:
            if keyword.lower() in name_str:
                # 예외 처리: 관광지로서의 역할이 있는 경우
                tourist_keywords = {'관광', '테마', '파크', '랜드', '월드', '호텔',
                                  '맛집', '식당', '카페', '박물관', '전시', '갤러리', '문화'}
                if any(tk in name_str for tk in tourist_keywords) and keyword != '아파트':
                    continue
                return True
        return False
        
    def process_visit_area_features(self, visit_area_df, id_to_index=None):
        """방문지 특성 처리 - 중복 ID 처리 포함"""
        visit_area_df = visit_area_df.copy()
        
        # 1. 유효한 NEW_VISIT_AREA_ID만 필터링
        if 'NEW_VISIT_AREA_ID' in visit_area_df.columns:
            valid_rows = visit_area_df['NEW_VISIT_AREA_ID'].notna()
            visit_area_df = visit_area_df[valid_rows].copy()
            logger.info(f"NEW_VISIT_AREA_ID가 있는 방문지: {len(visit_area_df)}개")
        else:
            logger.error("NEW_VISIT_AREA_ID 컬럼이 존재하지 않습니다!")
            raise ValueError("NEW_VISIT_AREA_ID 컬럼이 필요합니다.")
        
        # 2. ID 매핑이 제공된 경우, 해당 ID만 필터링하고 중복 처리
        if id_to_index is not None:
            visit_area_df['NEW_VISIT_AREA_ID'] = visit_area_df['NEW_VISIT_AREA_ID'].astype(int)
            
            # id_to_index에 있는 ID만 필터링
            valid_ids = set(id_to_index.keys())
            mask = visit_area_df['NEW_VISIT_AREA_ID'].isin(valid_ids)
            visit_area_df = visit_area_df[mask].copy()
            
            logger.info(f"ID 매핑에 있는 방문지: {len(visit_area_df)}개")
            
            # 🔧 중복 처리: 각 NEW_VISIT_AREA_ID별로 집계
            logger.info("중복된 방문지 ID 집계 중...")
            
            # 숫자형 컬럼들의 평균 계산
            numeric_cols = ['X_COORD', 'Y_COORD', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION']
            agg_dict = {}
            
            for col in numeric_cols:
                if col in visit_area_df.columns:
                    agg_dict[col] = 'mean'  # 평균값 사용
            
            # 범주형 컬럼들의 최빈값 또는 첫 번째 값 사용
            categorical_cols = ['VISIT_AREA_NM', 'VISIT_AREA_TYPE_CD', 'VISIT_CHC_REASON_CD']
            for col in categorical_cols:
                if col in visit_area_df.columns:
                    agg_dict[col] = 'first'  # 첫 번째 값 사용
            
            # 집계 수행
            aggregated_df = visit_area_df.groupby('NEW_VISIT_AREA_ID').agg(agg_dict).reset_index()
            
            logger.info(f"집계 후 유니크 방문지: {len(aggregated_df)}개")
            
            # id_to_index의 순서대로 정렬
            ordered_ids = sorted(id_to_index.keys())
            existing_ids = [id for id in ordered_ids if id in aggregated_df['NEW_VISIT_AREA_ID'].values]
            
            # 정렬된 순서로 재배열
            id_to_idx_map = {id: i for i, id in enumerate(aggregated_df['NEW_VISIT_AREA_ID'])}
            aggregated_df['sort_order'] = aggregated_df['NEW_VISIT_AREA_ID'].map(
                lambda x: existing_ids.index(x) if x in existing_ids else 999999
            )
            aggregated_df = aggregated_df.sort_values('sort_order').drop('sort_order', axis=1).reset_index(drop=True)
            
            visit_area_df = aggregated_df
            logger.info(f"최종 정렬된 방문지: {len(visit_area_df)}개")
        
        # 3. 기본 컬럼 존재 여부 확인 및 결측치 처리
        required_cols = ['X_COORD', 'Y_COORD', 'VISIT_AREA_NM']
        for col in required_cols:
            if col not in visit_area_df.columns:
                logger.warning(f"필수 컬럼 {col}이 없습니다. 기본값으로 대체합니다.")
                if col in ['X_COORD', 'Y_COORD']:
                    visit_area_df[col] = 0.0
                else:
                    visit_area_df[col] = '알 수 없음'
        
        # 좌표 결측치 처리
        visit_area_df['X_COORD'] = pd.to_numeric(visit_area_df['X_COORD'], errors='coerce')
        visit_area_df['Y_COORD'] = pd.to_numeric(visit_area_df['Y_COORD'], errors='coerce')
        visit_area_df['X_COORD'] = visit_area_df['X_COORD'].fillna(visit_area_df['X_COORD'].mean())
        visit_area_df['Y_COORD'] = visit_area_df['Y_COORD'].fillna(visit_area_df['Y_COORD'].mean())
        
        # VISIT_CHC_REASON_CD 처리
        if 'VISIT_CHC_REASON_CD' in visit_area_df.columns:
            visit_area_df['VISIT_CHC_REASON_CD'] = pd.to_numeric(visit_area_df['VISIT_CHC_REASON_CD'], errors='coerce').fillna(0)
        else:
            visit_area_df['VISIT_CHC_REASON_CD'] = 0
        
        features = visit_area_df[['X_COORD', 'Y_COORD']].copy()
        
        # One-hot encoding (안전하게 처리)
        if 'VISIT_AREA_TYPE_CD' in visit_area_df.columns:
            type_onehot = pd.get_dummies(visit_area_df['VISIT_AREA_TYPE_CD'], prefix='type')
        else:
            type_onehot = pd.DataFrame({'type_unknown': [1] * len(visit_area_df)})
            
        reason_onehot = pd.get_dummies(visit_area_df['VISIT_CHC_REASON_CD'], prefix='reason')
        
        # 만족도 점수 처리 (안전하게)
        satisfaction_cols = ['DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION']
        for col in satisfaction_cols:
            if col in visit_area_df.columns:
                visit_area_df[col] = pd.to_numeric(visit_area_df[col], errors='coerce').fillna(3)
            else:
                visit_area_df[col] = 3  # 기본값
            visit_area_df[f'{col}_norm'] = (visit_area_df[col] - 1) / 4.0
        
        # 인기도 점수
        visit_area_df['popularity_score'] = (
            visit_area_df['DGSTFN_norm'] * 0.4 + 
            visit_area_df['REVISIT_INTENTION_norm'] * 0.3 + 
            visit_area_df['RCMDTN_INTENTION_norm'] * 0.3
        )
        
        # 제외할 장소에 대한 페널티 추가
        exclude_penalty = visit_area_df['VISIT_AREA_NM'].apply(self.should_exclude_location).astype(float) * -0.5
        
        # 모든 특성 결합
        features = pd.concat([
            features, type_onehot, reason_onehot,
            visit_area_df[['DGSTFN_norm', 'REVISIT_INTENTION_norm', 'RCMDTN_INTENTION_norm', 'popularity_score']],
            pd.DataFrame({'exclude_penalty': exclude_penalty})
        ], axis=1)
        
        logger.info(f"방문지 특성 처리 완료: {features.shape}")
        return self.visit_scaler.fit_transform(features.values.astype(np.float32))
    
    def create_enhanced_edges(self, move_df, visit_area_df):
        """향상된 엣지 생성 - 방문지 데이터에 존재하는 ID만 사용"""
        edges = []
        edge_weights = []
        
        logger.info("엣지 생성 시작...")
        
        # 1. 방문지 데이터에서 실제 존재하는 유효한 NEW_VISIT_AREA_ID만 수집
        available_visit_ids = set()
        if 'NEW_VISIT_AREA_ID' in visit_area_df.columns:
            available_from_visit = visit_area_df['NEW_VISIT_AREA_ID'].dropna().astype(int)
            available_visit_ids.update(available_from_visit)
        
        logger.info(f"방문지 데이터에서 사용 가능한 ID: {len(available_visit_ids)}개")
        
        # 2. 이동 데이터에서 유효한 ID 수집 (단, 방문지 데이터에 존재하는 것만)
        valid_visit_ids = set()
        
        # START_VISIT_AREA_ID 확인
        if 'NEW_START_VISIT_AREA_ID' in move_df.columns:
            start_ids = move_df['NEW_START_VISIT_AREA_ID'].dropna().astype(int)
            # 방문지 데이터에 존재하는 것만 추가
            valid_start_ids = start_ids[start_ids.isin(available_visit_ids)]
            valid_visit_ids.update(valid_start_ids)
        
        # END_VISIT_AREA_ID 확인
        if 'NEW_END_VISIT_AREA_ID' in move_df.columns:
            end_ids = move_df['NEW_END_VISIT_AREA_ID'].dropna().astype(int)
            # 방문지 데이터에 존재하는 것만 추가
            valid_end_ids = end_ids[end_ids.isin(available_visit_ids)]
            valid_visit_ids.update(valid_end_ids)
        
        # 3. 최종 유효 ID 목록 (방문지 데이터와 이동 데이터 모두에 존재하는 것)
        final_valid_ids = available_visit_ids.intersection(valid_visit_ids)
        final_valid_ids = sorted(list(final_valid_ids))
        id_to_index = {visit_id: idx for idx, visit_id in enumerate(final_valid_ids)}
        
        logger.info(f"최종 사용할 방문지 ID: {len(final_valid_ids)}개")
        logger.info(f"방문지 ID 매핑 생성: {len(id_to_index)}개")
        
        # 4. 통계 추적 변수
        total_moves = 0
        valid_edges = 0
        invalid_from_id = 0
        invalid_to_id = 0
        missing_trip_id = 0
        
        # 5. 엣지 생성
        for travel_id, group in tqdm(move_df.groupby("TRAVEL_ID"), desc="Processing travel groups"):
            group = group.sort_values("TRIP_ID").reset_index(drop=True)
            
            for i in range(1, len(group)):
                total_moves += 1
                
                # 이전 행과 현재 행에서 방문지 ID 추출
                prev_row = group.loc[i-1]
                curr_row = group.loc[i]
                
                # 다양한 방법으로 FROM/TO ID 추출 시도
                from_id = None
                to_id = None
                
                # 방법 1: NEW_END_VISIT_AREA_ID 사용 (가장 우선)
                if pd.notna(prev_row.get('NEW_END_VISIT_AREA_ID')):
                    from_id = int(prev_row['NEW_END_VISIT_AREA_ID'])
                elif pd.notna(prev_row.get('NEW_START_VISIT_AREA_ID')):
                    from_id = int(prev_row['NEW_START_VISIT_AREA_ID'])
                
                if pd.notna(curr_row.get('NEW_END_VISIT_AREA_ID')):
                    to_id = int(curr_row['NEW_END_VISIT_AREA_ID'])
                elif pd.notna(curr_row.get('NEW_START_VISIT_AREA_ID')):
                    to_id = int(curr_row['NEW_START_VISIT_AREA_ID'])
                
                # ID 유효성 검사 및 인덱스 변환
                if from_id is None or to_id is None:
                    missing_trip_id += 1
                    continue
                
                if from_id not in id_to_index:
                    invalid_from_id += 1
                    continue
                    
                if to_id not in id_to_index:
                    invalid_to_id += 1
                    continue
                
                # 유효한 엣지인 경우 추가
                from_idx = id_to_index[from_id]
                to_idx = id_to_index[to_id]
                
                # 같은 노드로의 self-loop 제외
                if from_idx == to_idx:
                    continue
                
                duration = curr_row.get("DURATION_MINUTES", 0) if "DURATION_MINUTES" in curr_row else 0
                transport = curr_row.get("MVMN_CD_1", 0) if "MVMN_CD_1" in curr_row else 0
                
                edges.append([from_idx, to_idx, duration, transport])
                edge_weights.append(1.0)
                valid_edges += 1
        
        # 6. 통계 출력
        logger.info(f"엣지 생성 통계:")
        logger.info(f"  - 총 이동 시도: {total_moves}개")
        logger.info(f"  - 유효한 엣지: {valid_edges}개")
        logger.info(f"  - TRIP_ID 누락: {missing_trip_id}개")
        logger.info(f"  - 유효하지 않은 FROM_ID: {invalid_from_id}개")
        logger.info(f"  - 유효하지 않은 TO_ID: {invalid_to_id}개")
        
        if not edges:
            logger.warning("❌ 유효한 엣지가 생성되지 않았습니다!")
            return None, None, None
            
        edges_df = pd.DataFrame(edges, columns=["FROM_ID", "TO_ID", "DURATION_MINUTES", "MVMN_CD_1"])
        
        # 교통수단 분류
        edges_df["MVMN_TYPE"] = edges_df["MVMN_CD_1"].apply(
            lambda code: "drive" if code in [1,2,3] else "public" if code in [4,5,6,7,8,9,10,11,12,13,50] else "other"
        )
        edges_df["is_drive"] = (edges_df["MVMN_TYPE"] == "drive").astype(int)
        edges_df["is_public"] = (edges_df["MVMN_TYPE"] == "public").astype(int)
        edges_df["is_other"] = (edges_df["MVMN_TYPE"] == "other").astype(int)
        
        # 엣지 인덱스와 속성 생성
        edge_index = torch.tensor(edges_df[["FROM_ID", "TO_ID"]].values.T, dtype=torch.long)
        edge_attr = torch.tensor(np.column_stack([
            edges_df[["DURATION_MINUTES"]].fillna(0).values,
            edges_df[["is_drive", "is_public", "is_other"]].values,
            np.array(edge_weights).reshape(-1, 1)
        ]), dtype=torch.float32)
        
        logger.info(f"✅ 엣지 생성 완료:")
        logger.info(f"  - 최종 엣지 수: {len(edges)}개")
        logger.info(f"  - 노드 인덱스 범위: 0 ~ {len(id_to_index)-1}")
        logger.info(f"  - 엣지 인덱스 최대값: {edge_index.max().item()}")
        
        return edge_index, edge_attr, id_to_index

    def process_travel_context(self, travel_df):
        """여행 컨텍스트 처리"""
        logger.info("여행 컨텍스트 처리 시작...")
        
        # 필요한 컬럼들
        travel_feature_cols = [
            'TOTAL_COST_BINNED_ENCODED', 'WITH_PET', 'MONTH', 'DURATION',
            'MVMN_기타', 'MVMN_대중교통', 'MVMN_자가용',
            'TRAVEL_PURPOSE_1', 'TRAVEL_PURPOSE_2', 'TRAVEL_PURPOSE_3',
            'TRAVEL_PURPOSE_4', 'TRAVEL_PURPOSE_5', 'TRAVEL_PURPOSE_6',
            'TRAVEL_PURPOSE_7', 'TRAVEL_PURPOSE_8', 'TRAVEL_PURPOSE_9',
            'WHOWITH_2인여행', 'WHOWITH_가족여행', 'WHOWITH_기타',
            'WHOWITH_단독여행', 'WHOWITH_친구/지인 여행'
        ]
        
        # 누락된 컬럼들을 0으로 초기화
        for col in travel_feature_cols:
            if col not in travel_df.columns:
                travel_df[col] = 0
        
        travel_features = travel_df[travel_feature_cols].fillna(0)
        return self.travel_scaler.fit_transform(travel_features.values.astype(np.float32))


class ImprovedTravelGNN(nn.Module):
    """향상된 여행 추천 GNN 모델"""
    def __init__(self, in_channels, hidden_channels, out_channels, travel_context_dim, 
                 num_heads=4, dropout=0.2):
        super().__init__()
        
        # GAT 레이어들
        self.gat1 = GATConv(in_channels, hidden_channels // num_heads, 
                           heads=num_heads, dropout=dropout, concat=True, edge_dim=5)
        self.gat2 = GATConv(hidden_channels, hidden_channels // num_heads, 
                           heads=num_heads, dropout=dropout, concat=True, edge_dim=5)
        self.gat3 = GATConv(hidden_channels, out_channels, 
                           heads=1, dropout=dropout, concat=False, edge_dim=5)
        
        # Batch Normalization
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)
        self.bn3 = nn.BatchNorm1d(out_channels)
        
        # 여행 컨텍스트 인코더
        self.travel_encoder = nn.Sequential(
            nn.Linear(travel_context_dim, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, out_channels)
        )
        
        # 거리 기반 attention
        self.distance_attention = nn.Sequential(
            nn.Linear(2, hidden_channels // 2),
            nn.ReLU(),
            nn.Linear(hidden_channels // 2, 1),
            nn.Sigmoid()
        )
        
        # 융합 네트워크
        self.fusion_net = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )
        
        # 선호도 예측 헤드
        self.preference_head = nn.Sequential(
            nn.Linear(out_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels // 2, 1),
            nn.Sigmoid()
        )
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, data, travel_context, return_attention=False):
        x = data['visit_area'].x
        edge_index = data['visit_area', 'moved_to', 'visit_area'].edge_index
        edge_attr = data['visit_area', 'moved_to', 'visit_area'].edge_attr
        
        # 좌표 정보 추출
        coords = x[:, :2]
        
        # GAT 레이어들
        x1 = self.gat1(x, edge_index, edge_attr)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        x1 = self.dropout(x1)
        
        x2 = self.gat2(x1, edge_index, edge_attr)
        x2 = self.bn2(x2)
        x2 = F.relu(x2 + x1)  # Residual connection
        x2 = self.dropout(x2)
        
        graph_embedding = self.gat3(x2, edge_index, edge_attr)
        graph_embedding = self.bn3(graph_embedding)
        
        # 거리 기반 attention 적용
        distance_weights = self.distance_attention(coords)
        graph_embedding = graph_embedding * distance_weights
        
        # 여행 컨텍스트 처리
        travel_embedding = self.travel_encoder(travel_context)
        travel_embedding_expanded = travel_embedding.expand(graph_embedding.size(0), -1)
        
        # 특성 융합
        fused_features = torch.cat([graph_embedding, travel_embedding_expanded], dim=1)
        final_embedding = self.fusion_net(fused_features)
        
        # 선호도 점수 예측
        preference_scores = self.preference_head(final_embedding)
        
        if return_attention:
            return final_embedding, preference_scores, distance_weights
        
        return final_embedding, preference_scores


class TravelDatasetLoader:
    """여행 데이터셋 로더"""
    def __init__(self, processor):
        self.processor = processor
        
    def create_training_targets(self, visit_area_df, move_df, id_to_index):
        """학습 타겟 생성 - 중복 ID 고려한 집계"""
        logger.info("학습 타겟 생성 시작...")
        
        # 1. 유효한 방문지만 필터링
        valid_visit_df = visit_area_df[visit_area_df['NEW_VISIT_AREA_ID'].notna()].copy()
        valid_visit_df['NEW_VISIT_AREA_ID'] = valid_visit_df['NEW_VISIT_AREA_ID'].astype(int)
        
        # 2. 중복된 방문지 정보 집계
        logger.info("중복된 방문지 타겟 정보 집계 중...")
        
        # 만족도 관련 컬럼들의 평균 계산
        satisfaction_cols = ['DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION']
        agg_dict = {'VISIT_AREA_NM': 'first'}  # 방문지명은 첫 번째 값
        
        for col in satisfaction_cols:
            if col in valid_visit_df.columns:
                agg_dict[col] = 'mean'  # 만족도는 평균값
            else:
                valid_visit_df[col] = 3  # 기본값
                agg_dict[col] = 'mean'
        
        # 각 NEW_VISIT_AREA_ID별로 집계
        aggregated_visit_df = valid_visit_df.groupby('NEW_VISIT_AREA_ID').agg(agg_dict).reset_index()
        
        logger.info(f"집계된 유니크 방문지: {len(aggregated_visit_df)}개")
        
        # 3. 방문 빈도 계산 (실제 ID 기준, 유효한 것만)
        valid_move_df = move_df.copy()
        visit_counts = {}
        
        # END_VISIT_AREA_ID에서 방문 빈도 계산
        if 'NEW_END_VISIT_AREA_ID' in valid_move_df.columns:
            end_counts = valid_move_df['NEW_END_VISIT_AREA_ID'].dropna().astype(int).value_counts()
            visit_counts.update(end_counts.to_dict())
        
        # START_VISIT_AREA_ID에서도 방문 빈도 계산 (있다면)
        if 'NEW_START_VISIT_AREA_ID' in valid_move_df.columns:
            start_counts = valid_move_df['NEW_START_VISIT_AREA_ID'].dropna().astype(int).value_counts()
            for id, count in start_counts.items():
                visit_counts[id] = visit_counts.get(id, 0) + count
        
        max_visit_count = max(visit_counts.values()) if visit_counts else 1
        logger.info(f"방문 빈도 계산 완료: {len(visit_counts)}개 방문지, 최대 방문 {max_visit_count}회")
        
        # 4. 각 방문지에 대한 타겟 점수 계산 (id_to_index 순서로)
        targets = []
        missing_ids = []
        
        # aggregated_visit_df를 딕셔너리로 변환하여 빠른 접근
        visit_dict = aggregated_visit_df.set_index('NEW_VISIT_AREA_ID').to_dict('index')
        
        # id_to_index의 순서대로 타겟 생성
        for visit_id in sorted(id_to_index.keys()):
            if visit_id in visit_dict:
                row_data = visit_dict[visit_id]
                
                # 방문 빈도 (정규화)
                visit_freq = visit_counts.get(visit_id, 0)
                freq_score = min(visit_freq / max_visit_count, 1.0)
                
                # 만족도 점수 (안전하게 처리)
                dgstfn = row_data.get('DGSTFN', 3)
                revisit = row_data.get('REVISIT_INTENTION', 3)
                recommend = row_data.get('RCMDTN_INTENTION', 3)
                
                # None이나 NaN인 경우 기본값 사용
                dgstfn = dgstfn if pd.notna(dgstfn) else 3
                revisit = revisit if pd.notna(revisit) else 3
                recommend = recommend if pd.notna(recommend) else 3
                
                satisfaction = (dgstfn * 0.4 + revisit * 0.3 + recommend * 0.3) / 5.0
                
                # 제외 장소에 대한 페널티
                area_name = row_data.get('VISIT_AREA_NM', '')
                exclude_penalty = 0.3 if self.processor.should_exclude_location(area_name) else 0
                
                # 최종 타겟 점수
                final_score = (freq_score * 0.6 + satisfaction * 0.4) - exclude_penalty
                final_score = max(0, min(final_score, 1))  # 0-1 범위로 클램핑
                
                targets.append(final_score)
            else:
                # 해당 ID가 visit_area_df에 없는 경우
                missing_ids.append(visit_id)
                # 기본 점수: 방문 빈도만 고려
                visit_freq = visit_counts.get(visit_id, 0)
                freq_score = min(visit_freq / max_visit_count, 1.0)
                targets.append(freq_score * 0.5)  # 낮은 기본 점수
        
        if missing_ids:
            logger.warning(f"방문지 정보가 없는 ID {len(missing_ids)}개: {missing_ids[:10]}...")
        
        logger.info(f"타겟 생성 완료: {len(targets)}개")
        logger.info(f"타겟 점수 범위: {min(targets):.3f} ~ {max(targets):.3f}")
        
        return torch.tensor(targets, dtype=torch.float32)


class TravelRecommendationTrainer:
    """여행 추천 모델 학습기"""
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.processor = EnhancedDataProcessor()
        self.dataset_loader = TravelDatasetLoader(self.processor)
        
    def prepare_data(self, visit_area_df, move_df, travel_df=None):
        """데이터 준비 - 매핑 실패 및 예외 상황 고려"""
        logger.info("데이터 준비 시작...")
        
        try:
            # 1. 먼저 엣지 생성하여 ID 매핑 얻기
            edge_index, edge_attr, id_to_index = self.processor.create_enhanced_edges(move_df, visit_area_df)
            
            if edge_index is None or id_to_index is None or len(id_to_index) == 0:
                logger.error("엣지 생성에 실패했거나 유효한 ID 매핑이 없습니다!")
                return None, None, None, None
            
            # 2. ID 매핑을 사용하여 방문지 특성 처리
            visit_features = self.processor.process_visit_area_features(visit_area_df, id_to_index)
            
            if visit_features is None or len(visit_features) == 0:
                logger.error("방문지 특성 처리에 실패했습니다!")
                return None, None, None, None
            
            # 3. 노드 수와 ID 매핑 수가 일치하는지 확인
            expected_nodes = len(id_to_index)
            actual_nodes = visit_features.shape[0]
            
            if expected_nodes != actual_nodes:
                logger.error(f"노드 수 불일치: 예상 {expected_nodes}, 실제 {actual_nodes}")
                return None, None, None, None
            
            # 4. 헤테로 그래프 데이터 생성
            data = HeteroData()
            data['visit_area'].x = torch.tensor(visit_features, dtype=torch.float32)
            data['visit_area', 'moved_to', 'visit_area'].edge_index = edge_index
            data['visit_area', 'moved_to', 'visit_area'].edge_attr = edge_attr
            
            # 5. 학습 타겟 생성 (ID 매핑 전달)
            targets = self.dataset_loader.create_training_targets(visit_area_df, move_df, id_to_index)
            
            if targets is None or len(targets) == 0:
                logger.error("학습 타겟 생성에 실패했습니다!")
                return None, None, None, None
            
            # 6. 타겟 수와 노드 수 일치 확인
            if len(targets) != expected_nodes:
                logger.error(f"타겟 수 불일치: 예상 {expected_nodes}, 실제 {len(targets)}")
                return None, None, None, None
            
            # 7. 여행 컨텍스트 처리 (있는 경우)
            travel_contexts = None
            if travel_df is not None and not travel_df.empty:
                try:
                    travel_contexts = self.processor.process_travel_context(travel_df)
                except Exception as e:
                    logger.warning(f"여행 컨텍스트 처리 실패: {e}, 기본값 사용")
                    travel_contexts = None
            
            if travel_contexts is None:
                # 기본 여행 컨텍스트 생성
                logger.info("기본 여행 컨텍스트 생성...")
                travel_contexts = np.zeros((1, 21), dtype=np.float32)  # 21개 특성
                travel_contexts[0, 2] = 6  # MONTH = 6 (6월)
                travel_contexts[0, 3] = 2  # DURATION = 2일
                travel_contexts[0, 5] = 1  # MVMN_대중교통 = 1
                travel_contexts[0, 16] = 1  # WHOWITH_2인여행 = 1
            
            # 8. 최종 검증
            num_nodes = data['visit_area'].x.shape[0]
            num_edges = data['visit_area', 'moved_to', 'visit_area'].edge_index.shape[1]
            num_features = data['visit_area'].x.shape[1]
            
            # 엣지 인덱스가 노드 범위를 벗어나지 않는지 확인
            max_edge_idx = edge_index.max().item()
            if max_edge_idx >= num_nodes:
                logger.error(f"엣지 인덱스 오류: 최대 인덱스 {max_edge_idx} >= 노드 수 {num_nodes}")
                return None, None, None, None
            
            logger.info(f"✅ 데이터 준비 완료:")
            logger.info(f"  - 노드 수: {num_nodes}")
            logger.info(f"  - 특성 수: {num_features}")
            logger.info(f"  - 엣지 수: {num_edges}")
            logger.info(f"  - 여행 컨텍스트: {travel_contexts.shape}")
            logger.info(f"  - ID 매핑 수: {len(id_to_index)}")
            logger.info(f"  - 타겟 수: {len(targets)}")
            logger.info(f"  - 엣지 인덱스 범위: 0 ~ {max_edge_idx}")
            
            return data, targets, travel_contexts, id_to_index
            
        except Exception as e:
            logger.error(f"데이터 준비 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return None, None, None, None
    
    def train_model(self, data, targets, travel_contexts, epochs=200, lr=0.001, 
                   weight_decay=1e-4, save_path="./models/"):
        """모델 학습"""
        logger.info("모델 학습 시작...")
        
        # 디렉토리 생성
        os.makedirs(save_path, exist_ok=True)
        
        data = data.to(self.device)
        targets = targets.to(self.device)
        travel_contexts = torch.tensor(travel_contexts, dtype=torch.float32).to(self.device)
        
        # 모델 초기화
        in_channels = data['visit_area'].x.shape[1]
        hidden_channels = 256
        out_channels = 128
        travel_context_dim = travel_contexts.shape[1]
        
        model = ImprovedTravelGNN(
            in_channels=in_channels,
            hidden_channels=hidden_channels,
            out_channels=out_channels,
            travel_context_dim=travel_context_dim,
            num_heads=4,
            dropout=0.2
        ).to(self.device)
        
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        
        # 수정된 부분: verbose 매개변수 제거
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=20
        )
        
        # 현재 학습률 추적을 위한 변수
        current_lr = lr
        
        # 학습 루프
        model.train()
        best_loss = float('inf')
        patience_counter = 0
        max_patience = 50
        
        losses = []
        
        for epoch in tqdm(range(epochs), desc="Training"):
            optimizer.zero_grad()
            
            # 첫 번째 여행 컨텍스트 사용 (배치 처리 시뮬레이션)
            travel_context = travel_contexts[0:1]
            
            # Forward pass
            embeddings, preference_scores = model(data, travel_context)
            
            # 손실 계산 (MSE)
            loss = F.mse_loss(preference_scores.squeeze(), targets)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            losses.append(loss.item())
            
            # 학습률 스케줄링 (수정된 부분: 학습률 변화 수동 추적)
            old_lr = current_lr
            scheduler.step(loss)
            current_lr = optimizer.param_groups[0]['lr']
            
            # 학습률이 변경된 경우 로그 출력
            if current_lr != old_lr:
                logger.info(f"Epoch {epoch}: Learning rate reduced from {old_lr:.6f} to {current_lr:.6f}")
            
            # Early stopping
            if loss.item() < best_loss:
                best_loss = loss.item()
                patience_counter = 0
                
                # 최고 모델 저장
                model_config = {
                    'in_channels': in_channels,
                    'hidden_channels': hidden_channels,
                    'out_channels': out_channels,
                    'travel_context_dim': travel_context_dim,
                    'num_heads': 4,
                    'dropout': 0.2
                }
                
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'model_config': model_config,
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': epoch,
                    'loss': best_loss,
                    'losses': losses
                }, os.path.join(save_path, 'improved_travel_recommendation_model.pt'))
                
            else:
                patience_counter += 1
                
            if patience_counter >= max_patience:
                logger.info(f"Early stopping at epoch {epoch}")
                break
                
            # 로그 출력
            if epoch % 20 == 0:
                logger.info(f'Epoch {epoch:03d}, Loss: {loss.item():.4f}, Best: {best_loss:.4f}, LR: {current_lr:.6f}')
        
        logger.info(f'학습 완료! 최종 손실: {best_loss:.4f}')
        
        return model, losses
    
    def save_processed_data(self, visit_area_df, data, id_to_index, save_path="./pickle/"):
        """처리된 데이터 저장 - 기존 포맷 호환"""
        logger.info("처리된 데이터 저장 시작...")
        
        os.makedirs(save_path, exist_ok=True)
        
        # 기존 포맷에 맞춘 데이터 저장
        save_data = {
            'visit_area_df': visit_area_df,
            'graph_data': data.cpu(),  # CPU로 이동하여 저장
            'visit_scaler': self.processor.visit_scaler,
            'travel_scaler': self.processor.travel_scaler,
            'device': str(self.device),
            # 추가 정보 (기존 코드 호환성 유지하면서 새 기능 제공)
            'id_to_index': id_to_index,  # ID 매핑 정보
            'region_info': {
                'num_nodes': data['visit_area'].x.shape[0],
                'num_edges': data['visit_area', 'moved_to', 'visit_area'].edge_index.shape[1],
                'num_features': data['visit_area'].x.shape[1]
            }
        }
        
        with open(os.path.join(save_path, 'improved_travel_data.pkl'), 'wb') as f:
            pickle.dump(save_data, f)
        
        logger.info(f"데이터 저장 완료: {save_path}")
        logger.info(f"저장된 데이터 구조:")
        logger.info(f"  - visit_area_df: {visit_area_df.shape}")
        logger.info(f"  - graph_data: 노드 {save_data['region_info']['num_nodes']}개, 엣지 {save_data['region_info']['num_edges']}개")
        logger.info(f"  - visit_scaler: {type(self.processor.visit_scaler).__name__}")
        logger.info(f"  - travel_scaler: {type(self.processor.travel_scaler).__name__}")
        logger.info(f"  - id_to_index: {len(id_to_index)}개 매핑")
        logger.info(f"  - device: {self.device}")


def main_training_pipeline(visit_area_df, move_df, travel_df=None, 
                          model_save_path="./models/", data_save_path="./pickle/"):
    """전체 학습 파이프라인 - 매핑 실패 고려"""
    logger.info("=" * 60)
    logger.info("🚀 GNN 여행 추천 시스템 학습 시작!")
    logger.info("=" * 60)
    
    try:
        # 1. 트레이너 초기화
        trainer = TravelRecommendationTrainer()
        
        # 2. 데이터 준비
        data, targets, travel_contexts, id_to_index = trainer.prepare_data(visit_area_df, move_df, travel_df)
        
        if data is None or targets is None:
            logger.error("❌ 데이터 준비에 실패했습니다!")
            return None, None, None, None
        
        # 3. 모델 학습
        model, losses = trainer.train_model(
            data, targets, travel_contexts,
            epochs=200,
            lr=0.001,
            save_path=model_save_path
        )
        
        # 4. 처리된 데이터 저장
        trainer.save_processed_data(visit_area_df, data, id_to_index, save_path=data_save_path)
        
        # 5. 학습 결과 요약
        logger.info("\n" + "=" * 60)
        logger.info("✅ 학습 완료!")
        logger.info(f"📁 모델 저장 경로: {model_save_path}")
        logger.info(f"📁 데이터 저장 경로: {data_save_path}")
        logger.info(f"📊 최종 손실: {min(losses):.4f}")
        logger.info("=" * 60)
        
        return model, losses, data, targets
        
    except Exception as e:
        logger.error(f"❌ 학습 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None, None


def main_region_training(move_path, travel_path, visit_area_path, region_name):
    """지역별 모델 학습 및 저장 - 매핑 실패 및 데이터 검증 강화"""
    logger.info(f"\n{'='*60}")
    logger.info(f"🚀 지역 '{region_name}' GNN 모델 학습 시작!")
    logger.info(f"{'='*60}")
    
    try:
        # 1. 데이터 로드
        logger.info("📂 데이터 로드 중...")
        
        # 파일 존재 확인
        if not os.path.exists(move_path):
            logger.error(f"❌ 이동내역 파일을 찾을 수 없습니다: {move_path}")
            return False
        if not os.path.exists(visit_area_path):
            logger.error(f"❌ 방문지 파일을 찾을 수 없습니다: {visit_area_path}")
            return False
            
        # CSV 파일 로드
        move_df = pd.read_csv(move_path)
        visit_area_df = pd.read_csv(visit_area_path)
        
        # 2. 기본 데이터 검증
        logger.info("🔍 데이터 검증 중...")
        
        # 이동내역 검증
        if len(move_df) == 0:
            logger.error(f"❌ 지역 '{region_name}': 이동내역 데이터가 비어있습니다.")
            return False
            
        # 방문지 검증
        if len(visit_area_df) == 0:
            logger.error(f"❌ 지역 '{region_name}': 방문지 데이터가 비어있습니다.")
            return False
        
        # NEW_VISIT_AREA_ID 검증
        if 'NEW_VISIT_AREA_ID' not in visit_area_df.columns:
            logger.error(f"❌ 지역 '{region_name}': 방문지 데이터에 NEW_VISIT_AREA_ID 컬럼이 없습니다.")
            return False
            
        valid_visit_areas = visit_area_df['NEW_VISIT_AREA_ID'].notna().sum()
        if valid_visit_areas == 0:
            logger.error(f"❌ 지역 '{region_name}': 유효한 NEW_VISIT_AREA_ID가 없습니다.")
            return False
            
        # 이동내역의 NEW_END_VISIT_AREA_ID 검증
        has_end_id = 'NEW_END_VISIT_AREA_ID' in move_df.columns
        has_start_id = 'NEW_START_VISIT_AREA_ID' in move_df.columns
        
        if not has_end_id and not has_start_id:
            logger.error(f"❌ 지역 '{region_name}': 이동내역에 NEW_END_VISIT_AREA_ID 또는 NEW_START_VISIT_AREA_ID가 없습니다.")
            return False
        
        valid_moves = 0
        if has_end_id:
            valid_moves += move_df['NEW_END_VISIT_AREA_ID'].notna().sum()
        if has_start_id:
            valid_moves += move_df['NEW_START_VISIT_AREA_ID'].notna().sum()
            
        if valid_moves == 0:
            logger.error(f"❌ 지역 '{region_name}': 유효한 이동 기록이 없습니다.")
            return False
        
        # 여행 정보 파일 로드 (있는 경우)
        travel_df = None
        if os.path.exists(travel_path):
            travel_df = pd.read_csv(travel_path)
            logger.info(f"✅ 여행정보 데이터 로드: {len(travel_df)}개 레코드")
        else:
            logger.warning(f"⚠️ 여행정보 파일이 없습니다: {travel_path}")
        
        logger.info(f"✅ 데이터 로드 및 검증 완료:")
        logger.info(f"  - 이동내역: {len(move_df)}개 레코드 (유효 이동: {valid_moves}개)")
        logger.info(f"  - 방문지: {len(visit_area_df)}개 레코드 (유효 방문지: {valid_visit_areas}개)")
        
        # 3. 저장 경로 설정
        model_save_path = f"./models/{region_name}/"
        data_save_path = f"./pickle/{region_name}/"
        
        # 4. 트레이너 초기화
        trainer = TravelRecommendationTrainer()
        
        # 5. 데이터 준비
        logger.info("🔧 데이터 전처리 중...")
        data, targets, travel_contexts, id_to_index = trainer.prepare_data(visit_area_df, move_df, travel_df)
        
        # 6. 최종 데이터 검증
        if data is None or targets is None:
            logger.error(f"❌ 지역 '{region_name}': 데이터 전처리에 실패했습니다.")
            return False
        
        min_nodes = 10  # 최소 노드 수
        min_edges = 5   # 최소 엣지 수
        
        num_nodes = data['visit_area'].x.shape[0]
        num_edges = data['visit_area', 'moved_to', 'visit_area'].edge_index.shape[1]
        
        if num_nodes < min_nodes:
            logger.error(f"❌ 지역 '{region_name}': 노드 수가 너무 적습니다 ({num_nodes} < {min_nodes})")
            return False
            
        if num_edges < min_edges:
            logger.error(f"❌ 지역 '{region_name}': 엣지 수가 너무 적습니다 ({num_edges} < {min_edges})")
            return False
        
        # 7. 모델 학습
        logger.info("🎯 모델 학습 시작...")
        model, losses = trainer.train_model(
            data, targets, travel_contexts,
            epochs=200,
            lr=0.001,
            save_path=model_save_path
        )
        
        # 8. 처리된 데이터 저장
        logger.info("💾 데이터 저장 중...")
        trainer.save_processed_data(visit_area_df, data, id_to_index, save_path=data_save_path)
        
        # 9. 학습 결과 요약
        logger.info(f"\n{'='*60}")
        logger.info(f"✅ 지역 '{region_name}' 학습 완료!")
        logger.info(f"📁 모델 저장: {model_save_path}")
        logger.info(f"📁 데이터 저장: {data_save_path}")
        logger.info(f"📊 최종 손실: {min(losses):.4f}")
        logger.info(f"📊 총 에포크: {len(losses)}")
        logger.info(f"🎯 노드 수: {num_nodes}, 엣지 수: {num_edges}")
        logger.info(f"{'='*60}")
        
        return True
        
    except Exception as e:
        logger.error(f"❌ 지역 '{region_name}' 학습 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return False


def main():
    """지역별 모델 학습 메인 함수"""
    logger.info("🌍 전체 지역 GNN 모델 학습 시작!")
    
    base_dir = './merged_csv/merged_csv_region/'
    
    # 지역 목록 가져오기
    if not os.path.exists(base_dir):
        logger.error(f"❌ 기본 디렉토리를 찾을 수 없습니다: {base_dir}")
        return
    
    region_list = [r for r in os.listdir(base_dir) 
                   if not r.startswith('.') and os.path.isdir(os.path.join(base_dir, r))]
    
    if not region_list:
        logger.error(f"❌ 처리할 지역이 없습니다: {base_dir}")
        return
    
    logger.info(f"📍 총 {len(region_list)}개 지역 발견: {region_list}")
    
    # 성공/실패 추적
    success_regions = []
    failed_regions = []
    
    # 각 지역별로 처리
    for i, region_name in enumerate(region_list, 1):
        logger.info(f"\n{'🌟' * 20}")
        logger.info(f"📍 [{i}/{len(region_list)}] 지역 '{region_name}' 처리 시작!")
        logger.info(f"{'🌟' * 20}")
        
        region_path = os.path.join(base_dir, region_name)
        
        # 파일 경로 정의
        move_path = os.path.join(region_path, 'fin', '이동내역_fin.csv')
        travel_path = os.path.join(region_path, 'fin', '여행_fin.csv')
        visit_area_path = os.path.join(region_path, 'fin', '방문지_fin.csv')
        
        # 지역별 학습 실행
        success = main_region_training(
            move_path=move_path,
            travel_path=travel_path,
            visit_area_path=visit_area_path,
            region_name=region_name
        )
        
        if success:
            success_regions.append(region_name)
            logger.info(f"✅ 지역 '{region_name}' 처리 완료!")
        else:
            failed_regions.append(region_name)
            logger.error(f"❌ 지역 '{region_name}' 처리 실패!")
    
    # 최종 결과 요약
    logger.info(f"\n{'🏁' * 30}")
    logger.info("🏁 전체 지역 처리 완료!")
    logger.info(f"{'🏁' * 30}")
    logger.info(f"✅ 성공한 지역 ({len(success_regions)}개): {success_regions}")
    if failed_regions:
        logger.info(f"❌ 실패한 지역 ({len(failed_regions)}개): {failed_regions}")
    
    # 모델 파일 구조 출력
    logger.info(f"\n📂 저장된 파일 구조:")
    for region in success_regions:
        logger.info(f"  📁 {region}/")
        logger.info(f"    🤖 models/{region}/improved_travel_recommendation_model.pt")
        logger.info(f"    💾 pickle/{region}/improved_travel_data.pkl")


# 개별 지역 테스트를 위한 함수
def test_single_region(region_name):
    """단일 지역 테스트용 함수"""
    base_dir = './merged_csv/merged_csv_region/'
    region_path = os.path.join(base_dir, region_name)
    
    move_path = os.path.join(region_path, 'fin', '이동내역_fin.csv')
    travel_path = os.path.join(region_path, 'fin', '여행_fin.csv')
    visit_area_path = os.path.join(region_path, 'fin', '방문지_fin.csv')
    
    return main_region_training(
        move_path=move_path,
        travel_path=travel_path,
        visit_area_path=visit_area_path,
        region_name=region_name
    )


# 실행 부분
if __name__ == "__main__":
    # 전체 지역 학습 실행
    main()
    
    # 또는 특정 지역만 테스트하고 싶다면:
    # test_single_region("서울특별시")
    
    # 주의사항 정리
    # 1. 데이터 매핑 실패나 누락이 있어도 안전하게 처리됩니다
    # 2. 각 지역별로 최소 노드 수(10개)와 엣지 수(5개) 검증을 합니다
    # 3. 상세한 로그를 통해 문제점을 파악할 수 있습니다
    # 4. ID 매핑과 엣지 인덱스가 항상 일치하도록 보장됩니다

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-08 15:12:08,199 - INFO - 🌍 전체 지역 GNN 모델 학습 시작!
2025-06-08 15:12:08,200 - INFO - 📍 총 4개 지역 발견: ['서부권', '동부권', '제주도 및 도서지역', '수도권']
2025-06-08 15:12:08,200 - INFO - 
🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
2025-06-08 15:12:08,200 - INFO - 📍 [1/4] 지역 '서부권' 처리 시작!
2025-06-08 15:12:08,200 - INFO - 🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
2025-06-08 15:12:08,201 - INFO - 
2025-06-08 15:12:08,201 - INFO - 🚀 지역 '서부권' GNN 모델 학습 시작!
2025-06-08 15:12:08,201 - INFO - ============================================================
2025-06-08 15:12:08,201 - INFO - 📂 데이터 로드 중...
2025-06-08 15:12:08,365 - INFO - 🔍 데이터 검증 중...
2025-06-08 15:12:08,373 - INFO - ✅ 여행정보 데이터 로드: 3222개 레코드
2025-06-08 15:12:08,374 - INFO - ✅ 데이터 로드 및 검증 완료:
2025-06-08 15:12:08,374 - INFO -   - 이동내역: 36226개